In [1]:
import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import inv
from scipy.sparse.linalg import spsolve
from scipy import random

In [2]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)


# Poisson euqation

\begin{equation}
\int{\nabla u \ \nabla v dx} = \int{f \ v dx} 
\end{equation}



In [3]:
# Poisson problem with hom. Dirichlet BC
fes = H1(mesh, order=3,dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a_jac")
gfu_b = GridFunction(fes,name = "u_b_jac")
gfu_a_dir = GridFunction(fes,name = "u_a_dir")
gfu_b_dir = GridFunction(fes,name = "u_b_dir")
print(len(gfu_a.vec))

4342


In [4]:
def SolveCG(A, C, u_grid, f_grid, maxsteps:int=200, tol:float=1e-8, printrate=True, errConv=True):
    """My CG
    
    args:
    A... matrix of the BLF
    C... (inverse) matrix of the Preconditioner (NGSolve stores only the inverse of C)
    u_grid... vector of the solution gridfunction
    f_grid... vector of the RHS (LF)
    maxsteps... maximum number of iterations
    tol... (error) tolerance
    printrate... enables printing of iteration number and error per step
    """
    r = u_grid.CreateVector()
    d = u_grid.CreateVector()
    p = u_grid.CreateVector()
    dTp = u_grid.CreateVector()
    Ap = u_grid.CreateVector()
    dC = u_grid.CreateVector()
    pTAp = u_grid.CreateVector()
    tempN = u_grid.CreateVector()
    tempd_res = u_grid.CreateVector()
    #total_err = u_grid.CreateVector()
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    u_grid.data = Projector(fes.FreeDofs(), True) * r
    
    d.data = f_grid - A*u_grid
    p.data = C*d.data
    
    last_err = 0
    err_counter = 0
    
    for k in range(maxsteps):
        Ap.data = A * p.data
        dTp = InnerProduct(d.data, p.data)

        pTAp = InnerProduct(p.data, Ap)
        alpha = dTp/pTAp

        u_grid.data = u_grid.data + alpha*p.data
        
        tempd_res = d.data
        d.data = d.data - alpha*Ap.data
        total_err = InnerProduct(tempd_res,d.data)*InnerProduct(tempd_res,d.data)
        if printrate:
            print(f"(step: {k} ,error: {total_err})")
        if total_err < tol:
            print(f"Err tol reached: error={total_err}, tol={tol}")
            break
        if errConv and np.isclose(last_err, total_err, rtol=tol):
            if err_counter:
                print(f"Error did not improve: error={last_err}\n...terminating")
                break
            err_counter +=1
        last_err = total_err
        #print(total_err)
        dC.data = C*d.data
        #tempN = dC*Ap
        tempN = InnerProduct(dC, Ap)
        beta = -tempN/pTAp

        p.data = C*d.data + beta*p.data
    #gfu_new = u_grid  
    return u_grid  

In [5]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

Aa.Assemble()
Fa.Assemble()


c = Preconditioner(Aa,"local",test = True)
c.Update()
cdir = Preconditioner(Aa,"direct",test = True)
cdir.Update()

SolveCG(Aa.mat,c.mat,gfu_a.vec,Fa.vec, maxsteps=200, tol=1e-8, printrate=True)
SolveCG(Aa.mat,cdir.mat,gfu_a_dir.vec,Fa.vec, maxsteps=200, tol=1e-8, printrate=True)
#gif_a = GridFunction(fes,name = "gif_a")
#gif_a.vec.FV().NumPy()[:] = gfu_new
#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gfu_a)
Draw(gfu_a_dir)

(step: 0 ,error: 3.698749850652015e-10)
Err tol reached: error=3.698749850652015e-10, tol=1e-08
(step: 0 ,error: 2.127060738762445e-12)
Err tol reached: error=2.127060738762445e-12, tol=1e-08


In [6]:
ub = fes.TrialFunction()
vb = fes.TestFunction()

Ab = BilinearForm(fes)
Fb = LinearForm(fes)

Ab += grad(ub)*grad(vb)*dx

Fb += vb*dx

Ab.Assemble()
Fb.Assemble()


c = Preconditioner(Ab,"local",test = True)
c.Update()
cdir = Preconditioner(Ab, "direct",test = True)
cdir.Update()


SolveCG(Ab.mat, c.mat, gfu_b.vec, Fb.vec, maxsteps=2000, tol=1e-8, printrate=True, errConv = False)
SolveCG(Ab.mat, cdir.mat, gfu_b_dir.vec, Fb.vec, maxsteps=200, tol=1e-8, printrate=True)

#gif_b = GridFunction(fes,name = "gif_b")
#gif_b.vec.FV().NumPy()[:] = gfu_new
#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gfu_b)
Draw(gfu_b_dir)

(step: 0 ,error: 1602.3867232104208)
(step: 1 ,error: 862.5219286850765)
(step: 2 ,error: 130.36807967738392)
(step: 3 ,error: 118.42638317933918)
(step: 4 ,error: 39.53522342564604)
(step: 5 ,error: 39.87874014172776)
(step: 6 ,error: 18.737625873738914)
(step: 7 ,error: 19.055178321270883)
(step: 8 ,error: 10.946518677616499)
(step: 9 ,error: 10.969211532910531)
(step: 10 ,error: 7.179892403234559)
(step: 11 ,error: 7.093561961251638)
(step: 12 ,error: 5.060690917107733)
(step: 13 ,error: 4.956549946288113)
(step: 14 ,error: 3.7444474375528856)
(step: 15 ,error: 3.6527066566799857)
(step: 16 ,error: 2.8689777598498933)
(step: 17 ,error: 2.7955853654157212)
(step: 18 ,error: 2.256950741245248)
(step: 19 ,error: 2.2001993051830886)
(step: 20 ,error: 1.8126630107211785)
(step: 21 ,error: 1.769246944112058)
(step: 22 ,error: 1.4805091466079257)
(step: 23 ,error: 1.4473619729464784)
(step: 24 ,error: 1.2262604755626616)
(step: 25 ,error: 1.2009420467999226)
(step: 26 ,error: 1.02784448034